# Project 1

## Part 1: Basic Pathfinding
### Task choice 1: Depth-first Search

Represent the Maze: First, we need to represent the maze in a way that our program can understand. We will use a 2D array (or list of lists in Python), where each cell in the array corresponds to a cell in the maze. We will represent free spaces (where Pacman can move) as 0s, walls as 1s, the start position (Pacman's initial position) as 'P', and the goal (the dot) as '.'.

In [59]:
import os
class Maze:
    def __init__(self, maze):
        # Initialize the Maze object with the given maze
        self.maze = maze
        self.start = None
        self.goal = None
        self.free_spaces = []
        self.walls = []
        self.visited = []
        self.path = []
        self.backtracked = []
        self.directions = {}
        
        # Store the maze data in a 2d-array where:
        #      'P': the starting point
        #      '.': the goal point
        #      ' ': free space that can be navigated through
        #      '%': walls that cannot be navigated through
        for i in range(len(maze)):
            for j in range(len(maze[i])):
                if maze[i][j] == 'P':
                    self.start = (i, j)
                elif maze[i][j] == '.':
                    self.goal = (i, j)
                elif maze[i][j] == ' ':
                    self.free_spaces.append((i, j))
                elif maze[i][j] == '%':
                    self.walls.append((i, j))

    # Get the neighboring positions of a given position
    def get_neighbors(self, position):
        i, j = position
        neighbors = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
        return [pos for pos in neighbors if self.maze[pos[0]][pos[1]] in ['P', '.', ' '] and pos not in self.visited]

    # Used for directional result output
    def get_direction(self, pos1, pos2):
        # Get the direction from pos1 to pos2
        if pos2[0] > pos1[0]:
            return "↓"
        elif pos2[0] < pos1[0]:
            return "↑"
        elif pos2[1] > pos1[1]:
            return "→"
        elif pos2[1] < pos1[1]:
            return "←"
        else:
            # If the positions are the same, return the direction of the last movement
            return self.path[-1][1] if len(self.path) > 1 else None

    # =====================================================
    # Depth First Search (recursive)
    # 
    # Helper functions: get_direction, get_neighbors
    #
    # Author: Jacob Thieret
    #
    # Description: Recursively visits each neighboring position of the current position 
    #              until the goal is found or all possible paths have been exhausted. 
    #              Backtracking is performed by removing the current position from the path if 
    #              none of its neighbors lead to the goal. The process repeats until the goal is
    #              found or all positions have been visited.
    #
    # ====================================================
    def DepthFirstSearch(self, position=None):
        if position is None:
            position = self.start
        # Adds the position to the visited list.
        self.visited.append(position)
        
        # Determines the direction of movement from the previous position to the current one and adds it to the directions dictionary.
        if self.path:
            direction = self.get_direction(self.path[-1], position)
            self.directions[position] = direction
        
        # Adds the current position to the path.
        self.path.append(position)
        
        # Checks if the current position is the goal. If yes, it outputs a success message and returns True.
        if position == self.goal:
            print("Goal found: " + str(position))
            return True
        
        # If the current position is not the goal, it gets the list of unvisited neighboring positions and recursively calls DepthFirstSearch on each one.
        neighbors = self.get_neighbors(position)
        for neighbor in neighbors:
            if self.DepthFirstSearch(neighbor):
                return True
            
        # If none of the neighbors lead to the goal (i.e., DepthFirstSearch returns False for all of them), 
        # it removes the current position from the path (backtracking) and adds it to the backtracked list, then returns False
        self.path.pop()
        self.backtracked.append(position)
        return False
    
    # ===================================
    # Output Functions
    # ===================================

    # basic output
    # Writes the maze to a file, marking the visited positions with dots.
    def write_solution_dots(self, filename):
        solution_maze = [list(row) for row in self.maze]
        for i, j in self.visited:
            if solution_maze[i][j] == ' ':
                solution_maze[i][j] = '.'
        directory = os.path.dirname(filename)
        if directory:
            os.makedirs(directory, exist_ok=True)
        with open(filename, 'w') as file:
            for row in solution_maze:
                file.write(''.join(row) + '\n')
        
    # Directional Output [file]
    # Write the solved maze with directional arrows and bidirectional symbols to a file
    def write_solutionToFile(self, filename):
        solution_maze = [list(row) for row in self.maze]
        for position in self.visited:
            direction = self.directions.get(position, '.')
            if solution_maze[position[0]][position[1]] == ' ':
                solution_maze[position[0]][position[1]] = direction
        # replace the backtracked positions with bidirectional arrows to visually indicate multiple visits to a position
        for position in self.backtracked:
            if solution_maze[position[0]][position[1]] in ['→', '←']:
                solution_maze[position[0]][position[1]] = '↔'
            elif solution_maze[position[0]][position[1]] in ['↑', '↓']:
                solution_maze[position[0]][position[1]] = '↕'
        directory = os.path.dirname(filename)
        if directory:
            os.makedirs(directory, exist_ok=True)
        with open(filename, 'w', encoding='utf-8') as file:
            for row in solution_maze:
                file.write(''.join(row) + '\n')

    # Directional Output [console]
    # Write the solved maze with directional arrows and bidirectional symbols to a file
    def write_solutionToConsole(self, filename):
        solution_maze = [list(row) for row in self.maze]
        for position in self.visited:
            direction = self.directions.get(position, '.')
            if solution_maze[position[0]][position[1]] == ' ':
                solution_maze[position[0]][position[1]] = direction
        # replace the backtracked positions with bidirectional arrows to visually indicate multiple visits to a position
        for position in self.backtracked:
            if solution_maze[position[0]][position[1]] in ['→', '←']:
                solution_maze[position[0]][position[1]] = '↔'
            elif solution_maze[position[0]][position[1]] in ['↑', '↓']:
                solution_maze[position[0]][position[1]] = '↕'
        for row in solution_maze:
            print(''.join(row))

    # Print the number of steps taken and the path taken
    def print_path(self, b):
        # Number of steps taken
        print(f"Number of steps taken: {len(self.visited)}")
        print(f"Shortest path length found by DFS: {len(self.path)}")
        if b:
            print("Path taken:")
            for i in range(len(self.path)):
                # The start position
                if i == 0:
                    print(f"Started at {self.path[i]}")
                # Intermediate positions
                elif i < len(self.path) - 1:
                    direction = self.get_direction(self.path[i-1], self.path[i])
                    print(f"Moved {self.path[i-1]} {direction} {self.path[i]}")
                # The goal position
                else:
                    direction = self.get_direction(self.path[i-1], self.path[i])
                    print(f"Moved {direction} from {self.path[i-1]} to the goal at {self.path[i]}")          

# function to read in the maze
def read_maze_from_file(filename):
    with open(filename, 'r') as file:
        maze = [list(line.strip()) for line in file]
    return maze


In [60]:
smallMaze = read_maze_from_file('Maze/smallMaze.lay')

# Initialize small maze object and perform the DFS
smallMaze_obj = Maze(smallMaze)
smallMaze_obj.DepthFirstSearch()

# Write to a solution directional file, and print directional to console
smallMaze_obj.write_solutionToConsole()
smallMaze_obj.write_solutionToFile('Maze/solutions/smallMaze-solution.lay')
smallMaze_obj.write_solution_dots('Maze/solutions/smallMaze-solution-dots.lay')

# Print the step count, and step by step actions displayig the shortest path DFS found to the goal
smallMaze_obj.print_path(False)

Goal found: (8, 1)
%%%%%%%%%%%%%%%%%%%%%%
%↕%%↔↔↔↔↔↔↔↕% %      %
%↔↔↔↕%%%%%%↕% %%%%%% %
%%%%%%←←←←←P  %      %
%↕↔↔↔%↓%%%%%% %% %%%%%
%↕%%%%↓%↑→→→→    %   %
%↔↔↔↔↔↓→→%%%↓%%%   % %
%%%%%%%%%%←←↓ %%%%%% %
%.←←←←←←←←↓%%        %
%%%%%%%%%%%%%%%%%%%%%%
Number of steps taken: 54
Shortest path length found by DFS: 30


In [61]:
mediumMaze = read_maze_from_file('Maze/mediumMaze.lay')

# Initialize small maze object and perform the DFS
mediumMaze_obj = Maze(mediumMaze)
mediumMaze_obj.DepthFirstSearch()

# Write to a solution directional file, and print directional to console
mediumMaze_obj.write_solutionToConsole()
mediumMaze_obj.write_solutionToFile('Maze/solutions/mediumMaze-solution.lay')

# Print the step count, and step by step actions displayig the shortest path DFS found to the goal
mediumMaze_obj.print_path(False)

Goal found: (16, 1)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%↕↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔↔P%
%↕%%%%%%%%%%%%%%%%%%%%%%%↕%%%%%%%%↓%
%↕%%↔↔↕%↔↔↕%←←←←←↑%%%%%%%↕↔↔%%←←←←↓%
%↕%%↕%↕%↕%↕%↓%%%%↑%%%%%%%%%↕%%↓%%%%%
%↕%%↕%↕%↕%↕%↓→→  ←←←↑↔↔↔↕%%↕%%↓→→→→%
%↕%%↕%↕%↕%↕%↕%↓%%%%↕↑%%%↔↔↔↕%%%%%%↓%
%↕%↔↕%↕%↕%↔↔↕%↓→→→%%↑%%%%%%%% ←←←←↓%
%↕%%↕%↕%↕%%%%%%%%↓%%←←←←←←←↑%%↓%%%%%
%↕%%↕%↔↔↕%%←←←←←←↓%%%%%%%%%↑%%↓→→→→%
%↔↔↔↕%%%%%%↓%%%%%%%↑→→→→→%%↑%%%%%%↓%
%%%%%%←←←←←↓%↑→→→→→→%%%%↓%%↑% ←←←←↓%
%←←←←←↓%%%%%%↑%%%%%↕%←←←↓%%↑%%↓%%%%%
%↓%%%%%%↑→→→→→%←←←←←←↓%%%%%↑%%↓↑→→→%
%↓→→→→→→→%%%%%%↓%%%%%%%%%%%↑%%↓→%%↓%
%%%%%%%%%%←←←←←↓           ↑%%%%%%↓%
%.←←←←←←←←↓%%%%%%%%%%%%%%%%←←←←←←←↓%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Number of steps taken: 259
Shortest path length found by DFS: 165


In [62]:
bigMaze = read_maze_from_file('Maze/bigMaze.lay')

# Initialize small maze object and perform the DFS
bigMaze_obj = Maze(bigMaze)
bigMaze_obj.DepthFirstSearch()

# Write to a solution directional file, and print directional to console
bigMaze_obj.write_solutionToConsole()
bigMaze_obj.write_solutionToFile('Maze/solutions/bigMaze-solution.lay')

# Print the step count, and step by step actions displayig the shortest path DFS found to the goal
bigMaze_obj.print_path(False)

Goal found: (35, 1)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%↕↔↔↔↔↔↔% %↕%  ←←←←←←↑  %↕↔↔%↔↔↔↔↕%↕%
%↕%%%%%%% %↕%%%↓%↕%%%↑%%%↕%%%%%%%↕%↕%
%↔↔↔↔↕↔↔%  ←←←←↓%↕%  ↑  %↕↔↔↔↔%↕%↕↔↔%
%%%%%↕%%%%%↓%%%↕%↕% %↑%%%↕%%%%%↕%↕%%%
%↕↔↔%↕%↕%↕%↓  %↕%↕% %↑  %↕%↕↔↔%↕%↔↔↕%
%↕%%%↕%↕%↕%↓%%%↕%%%%%↑%%%↕%↕%%%↕%%%↕%
%↔↔↕↔↔↔↔%↔↔↓→→%↕↔↔%  ↑%↔↔↕↔↔%↕%↕%←←↑%
%%%↕%%%%%%%%%↓%%%%%%%↑%%%↕%%%↕%↕%↓%↑%
%  ←←←←←←←←←←↓%↑→→→→→→%↕%↕↔↔%←←←←↓%↑%
% %↓%%%%%↕%↕%%%↑%↕%↕%%%↕%↕%%%↓%%%↕%↑%
% %↓%↔↔↔↔↕%↕% %↑%↕%↕↔↔↔↔%↕↔↔%↓% %↕%↑%
% %↓%↕%%%%%%% %↑%%%%%%%%%↕%%%↓% %%%↑%
% %↓%↕%     %  ↑%↔↔↔↔↕%↔↔↕↔↔%↓  %  ↑%
%%%↓%%% % %%%%%↑%%%%%↕%%%↕%%%↓%%%%%↑%
%  ↓  % % %↔↔↑→→%↕%↔↔↕↔↔%↕%←←↓%↕%↕%↑%
% %↓% % % %%%↑%%%↕%%%↕%%%↕%↓%↕%↕%↕%↑%
% %↓% % %    ←←←←←←←←←←←←↑%↓%↕%↑→→→→%
%%%↓%%%%%%% % %↕%%%%%↕%%%↑%↓%%%↑%%%%%
%↔↔↓→→  % % % %↕↔↔↔↔%↕↔↔%←←↓  %↑%   %
%%%%%↓% % %%%%%%%%%↕%%%%%%%%%%%↑% %%%
%   %↓%           %↕%↕↔↔↔↔%↑→→%↑%   %
% %%%↓%%%%% %%%%%%%%%↕%%%%%↑%↓%↑%%% %
% %←←↓%      %↔↑→→→→→→%↑→→→→%↓→→    %
% %↓%↕%%%%% %%%↑%↕%↕%↓%↑%%%%%%%%%%%%%
% %↓%↕↔↔%     %↑%↕%↕%↓→→    %↔

In [63]:
openMaze = read_maze_from_file('Maze/openMaze.lay')

# Initialize small maze object and perform the DFS
openMaze_obj = Maze(openMaze)
openMaze_obj.DepthFirstSearch()

# Write to a solution directional file, and print directional to console
openMaze_obj.write_solutionToConsole()
openMaze_obj.write_solutionToFile('Maze/solutions/openMaze-solution.lay')

# Print the step count, and step by step actions displayig the shortest path DFS found to the goal
openMaze_obj.print_path(False)

Goal found: (21, 1)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%↔↔↔↔↔↔↔↔↔↔↔↔↔↕←↑←↑←↑←↑←↑←↑←↑←↑←↑←↑P%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓←↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓%↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓%↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓%↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓%↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓%↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓%↕↑↓↑↓↑↓↑↓↑↓↑↓↑↓%
%↔↔↔↔↔↔↔↔↔↔↔↕%↕↓↑↓↑↓%↕←↓←↓←↓←↓←↓←↓←↓%
%%%%%%%%%%%%%%↕↓↑↓↑↓%%%%%%%%%%%%%%%%%
%←↑←↑←↑←↑←↑←↑%↕↓↑↓↑↓                %
%↓↑↓↑↓↑↓↑↓↑↓↑%↕↓↑↓↑↓                %
%↓↑↓↑↓↑↓↑↓↑↓↑%↕↓↑↓↑↓                %
%↓↑↓↑↓↑↓↑↓↑↓↑←↑↓↑↓↑↓                %
%↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓                %
%↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓↑↓                %
%.←↓←↓←↓←↓←↓←↓←↓←↓←↓                %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Number of steps taken: 704
Shortest path length found by DFS: 391


# Part 2: Search With Multiple Goals

In [100]:
class MazeMG:
    def __init__(self, maze):
        # Initialize the Maze object with the given maze
        self.maze = maze
        self.start = None
        self.goals = []  # List of goals
        self.original_goals = []
        self.free_spaces = []
        self.walls = []
        self.visited = []
        self.path = []
        self.backtracked = []
        self.directions = {}
        self.goal_counter = 1  # Initialize goal counter
        
        # Store the maze data in a 2d-array
        for i in range(len(maze)):
            for j in range(len(maze[i])):
                if maze[i][j] == 'P':
                    self.start = (i, j)
                elif maze[i][j] == '.':
                    self.goals.append((i, j))  # Add goal to list of goals
                    self.original_goals.append((i,j)) # Static goals list to reference goals later
                elif maze[i][j] == ' ':
                    self.free_spaces.append((i, j))
                elif maze[i][j] == '%':
                    self.walls.append((i, j))

    
    # Get the neighboring positions of a given position
    def get_neighbors(self, position):
        i, j = position
        neighbors = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
        return [pos for pos in neighbors if self.maze[pos[0]][pos[1]] in ['P', '.', ' '] and pos not in self.visited]

    # Used for directional result output
    def get_direction(self, pos1, pos2):
        # Get the direction from pos1 to pos2
        if pos2[0] > pos1[0]:
            return "↓"
        elif pos2[0] < pos1[0]:
            return "↑"
        elif pos2[1] > pos1[1]:
            return "→"
        elif pos2[1] < pos1[1]:
            return "←"
        else:
            # If the positions are the same, return the direction of the last movement
            return self.path[-1][1] if len(self.path) > 1 else None

    # =====================================================
    # Depth First Search (recursive)
    # 
    # Helper functions: get_direction, get_neighbors
    #
    # Author: Jacob Thieret
    #
    # Description: Recursively visits each neighboring position of the current position 
    #              until the goal is found or all possible paths have been exhausted. 
    #              Backtracking is performed by removing the current position from the path if 
    #              none of its neighbors lead to the goal. The process repeats until the goal is
    #              found or all positions have been visited.
    #
    # ====================================================
    def DepthFirstSearch(self, position=None):
        if position is None:
            position = self.start
        # Adds the position to the visited list.
        self.visited.append(position)
        
        # Determines the direction of movement from the previous position to the current one and adds it to the directions dictionary.
        if self.path:
            direction = self.get_direction(self.path[-1], position)
            self.directions[position] = direction
        
        # Adds the current position to the path.
        self.path.append(position)
        
        # Checks if the current position is a goal. If yes, it updates the maze, outputs a success message, and increments the goal counter.
        if position in self.goals:
            self.maze[position[0]][position[1]] = str(self.goal_counter)  # Update maze with goal counter
            print(f"Goal {self.goal_counter} found: {position}")
            self.goal_counter += 1  # Increment goal counter
            self.goals.remove(position)  # Remove the goal from the list of goals
        
        # It gets the list of unvisited neighboring positions and recursively calls DepthFirstSearch on each one.
        neighbors = self.get_neighbors(position)
        for neighbor in neighbors:
            self.DepthFirstSearch(neighbor)
            
        # If none of the neighbors lead to a goal (i.e., DepthFirstSearch returns False for all of them), 
        # it removes the current position from the path (backtracking) and adds it to the backtracked list.
        if position not in self.goals:  # Only backtrack if the current position is not a goal
            self.path.pop()
            self.backtracked.append(position)
    
    # ===================================
    # Output Functions
    # ===================================
        
    # Directional Output [file]
    # Write the solved maze with directional arrows and bidirectional symbols to a file
    def write_solutionToFile(self, filename):
        solution_maze = [list(row) for row in self.maze]
        for position in self.visited:
            direction = self.directions.get(position, '.')
            if solution_maze[position[0]][position[1]] == ' ':
                solution_maze[position[0]][position[1]] = direction
        # replace the backtracked positions with bidirectional arrows to visually indicate multiple visits to a position
        for position in self.backtracked:
            if solution_maze[position[0]][position[1]] in ['→', '←']:
                solution_maze[position[0]][position[1]] = '↔'
            elif solution_maze[position[0]][position[1]] in ['↑', '↓']:
                solution_maze[position[0]][position[1]] = '↕'
        directory = os.path.dirname(filename)
        if directory:
            os.makedirs(directory, exist_ok=True)
        with open(filename, 'w', encoding='utf-8') as file:
            for row in solution_maze:
                file.write(''.join(row) + '\n')

    def write_solutionToConsoleDotGoal(self):
        solution_maze = [list(row) for row in self.maze]
        for position in self.visited:
            direction = self.directions.get(position, '.')
            if solution_maze[position[0]][position[1]] == ' ':
                solution_maze[position[0]][position[1]] = direction
        
        # replace the numbered goals with their dots for better output formatting
        for goal in self.original_goals:
            solution_maze[goal[0]][goal[1]] = '.'
        
        # replace the backtracked positions with bidirectional arrows to visually indicate multiple visits to a position
        for position in self.backtracked:
            if solution_maze[position[0]][position[1]] in ['→', '←']:
                solution_maze[position[0]][position[1]] = '↔'
            elif solution_maze[position[0]][position[1]] in ['↑', '↓']:
                solution_maze[position[0]][position[1]] = '↕'
        for row in solution_maze:
            print(''.join(row))
    
    # Directional Output [console]
    # Write the solved maze with directional arrows and bidirectional symbols to a file
    def write_solutionToConsoleNumGoal(self):
        solution_maze = [list(row) for row in self.maze]
        for position in self.visited:
            direction = self.directions.get(position, '.')
            if solution_maze[position[0]][position[1]] == ' ':
                solution_maze[position[0]][position[1]] = direction
        # replace the backtracked positions with bidirectional arrows to visually indicate multiple visits to a position
        for position in self.backtracked:
            if solution_maze[position[0]][position[1]] in ['→', '←']:
                solution_maze[position[0]][position[1]] = '↔'
            elif solution_maze[position[0]][position[1]] in ['↑', '↓']:
                solution_maze[position[0]][position[1]] = '↕'
        for row in solution_maze:
            print(''.join(row))

    # Print the number of steps taken and the path taken
    def print_path(self):
        # Number of steps taken
        print(f"Number of steps taken: {len(self.visited)}")    

In [109]:
tinySearch = read_maze_from_file('Maze/tinySearch.lay')

# Initialize tiny search maze object and perform the DFS
tinySearch_obj = MazeMG(tinySearch)
tinySearch_obj.DepthFirstSearch()

# Write to a solution directional file, and print directional to console
print("\nOutput with goals in order found")
tinySearch_obj.write_solutionToConsoleNumGoal()
print("\nOutput with goals as dots")
tinySearch_obj.write_solutionToConsoleDotGoal()
tinySearch_obj.write_solutionToFile('Maze/solutions/tinySearch-solution.lay')

# Print the step count
tinySearch_obj.print_path()

Goal 1 found: (2, 4)
Goal 2 found: (1, 2)
Goal 3 found: (1, 1)
Goal 4 found: (1, 6)
Goal 5 found: (1, 7)
Goal 6 found: (4, 7)
Goal 7 found: (5, 7)
Goal 8 found: (5, 3)
Goal 9 found: (4, 1)
Goal 10 found: (5, 1)

Output with goals in order found
%%%%%%%%%
%32↔↕↔45%
%%%%1%%↕%
%↔↔↔P↔↔↕%
%9%%↕%%6%
%10%8↔↔↔7%
%%%%%%%%%

Output with goals as dots
%%%%%%%%%
%..↔↕↔..%
%%%%.%%↕%
%↔↔↔P↔↔↕%
%.%%↕%%.%
%.%.↔↔↔.%
%%%%%%%%%
Number of steps taken: 27


In [110]:
smallSearch = read_maze_from_file('Maze/smallSearch.lay')

# Initialize small search maze object and perform the DFS
smallSearch_obj = MazeMG(smallSearch)
smallSearch_obj.DepthFirstSearch()

# Write to a solution directional file, and print directional to console

print("\nOutput with goals in order found")
smallSearch_obj.write_solutionToConsoleNumGoal()
print("\nOutput with goals as dots")
smallSearch_obj.write_solutionToConsoleDotGoal()
smallSearch_obj.write_solutionToFile('Maze/solutions/smallSearch-solution.lay')

# Print the step count
smallSearch_obj.print_path()

Goal 1 found: (2, 13)
Goal 2 found: (1, 13)
Goal 3 found: (2, 10)
Goal 4 found: (3, 10)
Goal 5 found: (3, 9)
Goal 6 found: (3, 8)
Goal 7 found: (3, 7)
Goal 8 found: (2, 7)
Goal 9 found: (2, 4)
Goal 10 found: (1, 1)
Goal 11 found: (2, 1)
Goal 12 found: (3, 6)
Goal 13 found: (1, 14)
Goal 14 found: (1, 15)
Goal 15 found: (1, 18)
Goal 16 found: (2, 18)
Goal 17 found: (3, 18)

Output with goals in order found
%%%%%%%%%%%%%%%%%%%%
%10↔↔↔↔↔↕↔↔↔↔↔21314P↔15%
%11%%9%%8%%3%%1%%↕%16%
%↕%%↕%127654↔↔↔↔↔↕%17%
%%%%%%%%%%%%%%%%%%%%

Output with goals as dots
%%%%%%%%%%%%%%%%%%%%
%.↔↔↔↔↔↕↔↔↔↔↔...P↔.%
%.%%.%%.%%.%%.%%↕%.%
%↕%%↕%.....↔↔↔↔↔↕%.%
%%%%%%%%%%%%%%%%%%%%
Number of steps taken: 42


In [111]:
trickySearch = read_maze_from_file('Maze/trickySearch.lay')

# Initialize small search maze object and perform the DFS
trickySearch_obj = MazeMG(trickySearch)
trickySearch_obj.DepthFirstSearch()

# Write to a solution directional file, and print directional to console

print("\nOutput with goals in order found")
trickySearch_obj.write_solutionToConsoleNumGoal()
print("\nOutput with goals as dots")
trickySearch_obj.write_solutionToConsoleDotGoal()
trickySearch_obj.write_solutionToFile('Maze/solutions/trickySearch_obj-solution.lay')

# Print the step count
trickySearch_obj.print_path()

Goal 1 found: (2, 7)
Goal 2 found: (2, 4)
Goal 3 found: (2, 1)
Goal 4 found: (1, 1)
Goal 5 found: (2, 10)
Goal 6 found: (2, 13)
Goal 7 found: (1, 13)
Goal 8 found: (1, 14)
Goal 9 found: (5, 5)
Goal 10 found: (5, 4)
Goal 11 found: (5, 3)
Goal 12 found: (5, 2)
Goal 13 found: (5, 1)

Output with goals in order found
%%%%%%%%%%%%%%%%%%%%
%4↔↔↔↔↔↕↔↔↔↔↔78%↕↔↔%
%3%%2%%1%%5%%6%%↕%↕%
%↔↔↔↕↔↔↔↔P↔↔↔↔↔↔↔%↕%
%%%%%%%%%%%%%%%%%%↕%
%131211109↔↔↔↔↔↔↔↔↔↔↔↔↕%
%%%%%%%%%%%%%%%%%%%%

Output with goals as dots
%%%%%%%%%%%%%%%%%%%%
%.↔↔↔↔↔↕↔↔↔↔↔..%↕↔↔%
%.%%.%%.%%.%%.%%↕%↕%
%↔↔↔↕↔↔↔↔P↔↔↔↔↔↔↔%↕%
%%%%%%%%%%%%%%%%%%↕%
%.....↔↔↔↔↔↔↔↔↔↔↔↔↕%
%%%%%%%%%%%%%%%%%%%%
Number of steps taken: 64
